# Assignment 2: Task 2

โดย

ตุลธร วงศ์ชัย รหัสนักศึกษา 63070224

ธนภัทร ธีรรัตตัญญู รหัสนักศึกษา 63070227

> **README**:
> - 'Run all' is not recommended. 
> - Section 1 must be run before any other section.
> - Read and follow instruction of each section carefully if it is provided.


# 1. Init Project

> **Instruction**: Set file path of `traffy_fondue_tickets.csv` in section 1.2 before running this section.

## 1.1 Import Libraries and Packages

In [2]:
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

## 1.2 Import Dataset

In [3]:
raw_data = pd.read_csv(f'{os.getcwd()}/traffy_fondue_tickets.csv')
raw_data.head()

,running_id,issues,การเดินทาง,กีดขวาง,คนจรจัด,คลอง,ความปลอดภัย,ความสะอาด,จราจร,ต้นไม้,...,ป้ายจราจร,ร้องเรียน,สอบถาม,สะพาน,สัตว์จรจัด,สายไฟ,ห้องน้ำ,เสนอแนะ,เสียงรบกวน,แสงสว่าง
0,1,ทำเรื่องรับสมุดทหารกองหนุน หลังจบแล้วมีให้เสีย...,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,2,ท่อประปาแตกหน้าบ้าน บ้านเลขที่126 ซ.สุขสวัสดิ์...,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,3,ขอแจ้งเหตุถนนเป็นหลุม ที่บริเวณหน้าหมู่บ้าน ca...,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,โครงการเพอร์เฟคเพลสพระราม9กรุงเทพกรีฑา ใช้รถน้...,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
4,5,เคยแจ้งทาง เขตแล้วให้ช่วยเก็บตามบ้าน แต่ละซอย ...,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


# 2. Text Processing

In [45]:
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import word_tokenize

In [46]:
text = raw_data.loc[1000, 'issues']

In [5]:
STOP_WORDS = thai_stopwords()

In [49]:
def thai_word_tokenizer(text:str, engine:str):
    tokens = word_tokenize(text, engine=engine)

    tokens_to_return = list()
    #filter re
    for token in tokens:
        #filter number
        if (re.match(r"([\u0E00-\u0E7Fa-zA-Z])+", token) and (token not in STOP_WORDS)):
            tokens_to_return.append(token)
    return tokens_to_return


In [50]:
def newmm_tokenizer(text):
    return thai_word_tokenizer(text, engine='newmm')

def attacut_tokenizer(text):
    return thai_word_tokenizer(text, engine='attacut')

In [51]:
tokenizers = [newmm_tokenizer, attacut_tokenizer]

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [53]:
contents = raw_data['issues']
X_tfidf = dict()

for tokenizer in tokenizers:
  print(f'tokenizing docs with {tokenizer.__name__}')
  vectorizer = TfidfVectorizer(tokenizer=tokenizer, min_df=30, max_df=0.99)
  term_weighted = vectorizer.fit_transform(contents)
  X_tfidf[tokenizer.__name__] = term_weighted

tokenizing docs with newmm_tokenizer
tokenizing docs with attacut_tokenizer


KeyboardInterrupt: 

In [9]:
contents = raw_data['issues']

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer=newmm_tokenizer, min_df=30, max_df=0.99)
term_weighted = vectorizer.fit_transform(contents)

In [11]:
from sklearn.model_selection import train_test_split, GridSearchCV

X_train, X_test, y_train, y_test= train_test_split(term_weighted.toarray(), raw_data.iloc[:, 2:])

In [32]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [28]:
models = [
    {
        "model": KNeighborsClassifier(),
        "params": {"n_neighbors": list(range(3, 16, 2))}
    },
    {
        "model": RidgeClassifier(copy_X=True),
        "params": {
            "alpha": list(np.arange(0.1, 1.1, 0.1))
        }
    },
    {
        "model": RandomForestClassifier(),
        "params": {
            "n_estimators": list(range(25, 201, 25)),
            "min_samples_split": list(range(5, 26, 5)),
            "random_state": [28]
        }
    },
]

In [29]:
clf = GridSearchCV(model, param_grid=params, cv=5, n_jobs=-1, verbose=1)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'n_estimators': [25, 50, 75, 100, 125, 150, 175, 200]},
             verbose=1)

In [38]:
clf.best_params_

{'n_estimators': 175}

In [31]:
clf.best_score_

0.32324313710373004